# CRB paper

In [22]:
import sys
from pathlib import Path
depth = '../'
for i in range(5):
    sys.path.append(depth)
    depth += '../'
import os
from os.path import isfile, join
import importlib
import numpy as np
import torch
import scipy as sc
import scipy.io
import pickle
import MRF
from MRF.Training_parameters import *
from MRF.BaseModel import *
from MRF.Projection_complex import *
from MRF.models import *
from MRF.Offline import Network, Data_class, Performances
from MRF.Training_parameters import *
from MRF.simulate_signal import simulation
import matplotlib.pyplot as plt
from scipy.io import loadmat
import argparse
root = '../../../../'

In [55]:
test_slice = sc.io.loadmat(root+'paper_data/invivo_test_slice.mat')['x']
np.sum(test_slice)

7.011813

In [42]:
import wget
url = "https://drive.google.com/file/d/1VfN4PK06M4PSeV5t92m2FutReUafP8Rj/view?usp=sharing"
wget.download(url, root+'paper_data/network')


-1 / unknown

'../../../../paper_data/network'

In [44]:
network_name = 'full_joint_18_deep3_blochsimv3p2_complexproj_0817_noprojection'
name = 'd0520_v3_nonsweep_varyB0B1_complex_nob1_blochsim_R13_p3_lr_0.01_noise0.01_1000_1024_largerval_train'
data_file = 'paper_data/invivo_test_slice.mat'
name_predix = 'same_as_training'
compressed = 1
number_parameter_predict = 3


model = importlib.import_module('MRF.models.' + network_name)

save_root='/NN_recon/'
if not os.path.exists(save_root):
    os.makedirs(save_root)

print('name:')
print(name_predix)
print('root:')
print(root)
print(name_predix)
names = [name]



# data_files = ['x_mid1653_reg0_R_9_network_large_R_T2s_v1_noise_0.005_9_dim_CRB_loss_3_parameters_0724.mat',
#              'x_mid1653_reg0_R_9_network_large_R_T2s_v1_pd_scaling_noise_0.005_9_dim_CRB_loss_3_parameters_0724.mat',
#               'x_mid1653_reg0_R_9_network_large_R_T2s_v2_noise_0.005_9_dim_CRB_loss_3_parameters_0724.mat',
#               'x_mid1653_reg0_R_9_network_large_R_T2s_v2_pd_scaling_noise_0.005_9_dim_CRB_loss_3_parameters_0724.mat']

data_files = [data_file]

t_1 = time.time()
# total_params_sequence = []
for name, data_file in zip(names, data_files):
    # filepath = 'x_PCA_3.mat'
    # filepath = "/Users/liukangning/downloads/code-MRF-april20/invivo/x_mid1653_reg7e-06_R_10_Kangning_T1.mat"

    filepath = os.path.join(root, data_file)
    with open(root+'settings_files_offline/settings_'+name+'.pkl', 'rb') as f:
        settings = pickle.load(f)
        net = torch.load(join(root,'paper_data/network_d0520_v3_nonsweep_varyB0B1_complex_nob1_blochsim_R13_p3_lr_0.01_noise0.01_1000_1024_largerval_train'),map_location='cpu')
        training_parameters = Training_parameters(settings['batch_size'], 1, settings['nb_epochs'], settings['params'], settings['normalization'])
        projection = Projection(settings['start_by_projection'], settings['dimension_projection'], settings['initialization'], settings['normalization'], settings['namepca'])
        data_class = Data_class(training_parameters, settings['noise_type'], settings['noise_level'],
                                           settings['minPD'], settings['maxPD'], settings['nb_files'], settings['path_files'])
        validation_settings = {'validation': settings['validation'],'small_validation_size': settings['small_validation_size'], 'validation_size': settings['validation_size']}
        netw = model.model(projection=projection,nb_params=len(settings['params']))
        device = torch.device('cpu')
        netw.load_state_dict(net['NN'])
        netw.eval()

        import h5py
        import numpy as np
        try:
            from scipy.io import loadmat
            arrays = loadmat(filepath)
            fingers = arrays['x']
            fingers = fingers.T  # (18, 128, 192, 192)
        except:
            arrays = {}
            f = h5py.File(filepath, 'r')
            for k, v in f.items():
                 arrays[k] = np.array(v)
            fingers = arrays['x']
        t_2 = time.time()
        projection.initialization = 'Fixlayer'
        netwproj = model.model(projection=projection,nb_params=len(settings['params']))
        device = torch.device('cpu')
        dico = net['NN']
        if compressed:
            print('compressed training: nothing to delete for 1st layer')
        else:
            if net['initialization'] != 'Fixlayer':
                try:
                    del dico['fc1.weight']
                except:
                    del dico['fc1_real.weight']
                    del dico['fc1_imag.weight']
        print(fingers.shape)
        mrfshape = fingers.shape
        netwproj.load_state_dict(dico, strict=False)
        netwproj.eval()
        with torch.no_grad():
            fings = fingers.reshape((-1, mrfshape[1],mrfshape[2] * mrfshape[3]))
            sequence_to_stack = []
            for i in range(mrfshape[1]):
                fings_tmp = fings[:,i,:].T
                params_tmp = netwproj(torch.tensor(fings_tmp, dtype=torch.float))
                params_tmp = np.array(params_tmp)
                for ii, para in enumerate(settings['params']):
                    if settings['loss'][para] == 'MSE-Log':
                        params_tmp[:, ii] = 10 ** params_tmp[:, ii]
                params_tmp = params_tmp.reshape((mrfshape[2],mrfshape[3],number_parameter_predict))
                sequence_to_stack.append(params_tmp)
            params = np.stack(sequence_to_stack,axis=0)   # 128*192*192*3 (26, 156, 192, 256) * 3
        processing_time = time.time() - t_2
        params = np.moveaxis(params, [0, 2], [2, 0])
        print(params.shape)  # (215, 215, 170, 3)
        # total_params_sequence.append(params)
        # plt.imshow(params)
        # sc.io.savemat('2params-'+filepath, {'parameters': params})
        sc.io.savemat(os.path.join(save_root, 'qM_'+data_file[2:-4]+ '_'+name_predix+'.mat'), {'qM': params})
        total_time = time.time() - t_1
        print(os.path.join(save_root, 'qM_'+data_file[2:-4]+ '_'+name_predix+'.mat'))
        print('done results')
        print('processing_time')
        print(processing_time)
        print('total_time')
        print(total_time)
        
        
        
        
        
        #
        # with torch.no_grad():
        #
        #     # try:#212   192   160 #256 192 162
        #     #     fings = fingers.reshape((18,128,192*192))
        #     # except:
        #     #     fings = fingers.reshape((9, 128, 192 * 192))
        #     # print('fingers.shape')
        #     # print(fingers.shape) #(26, 160, 192, 212)
        #     fings = -fingers.reshape((-1, mrfshape[1], mrfshape[2] * mrfshape[3]))
        #     sequence_to_stack = []
        #     for i in range(mrfshape[1]):
        #         fings_tmp = fings[:, i, :].T
        #         params_tmp = netwproj(torch.tensor(fings_tmp, dtype=torch.float))
        #
        #         params_tmp = np.array(params_tmp)
        #
        #         params_tmp = params_tmp.reshape((mrfshape[2], mrfshape[3], 3))
        #         sequence_to_stack.append(params_tmp)
        #     params = np.stack(sequence_to_stack, axis=0)  # 128*192*192*3 (26, 156, 192, 256) * 3
        #     params = np.moveaxis(params, [0, 2], [2, 0])
        #     print(params.shape)  # (215, 215, 170, 3)
        #     # total_params_sequence.append(params)
        #     # plt.imshow(params)
        #     # sc.io.savemat('2params-'+filepath, {'parameters': params})
        # sc.io.savemat(os.path.join(save_root, 'qM_'+data_file[:-4] + 'results' + '_' + args.name_predix + '_reverse.mat'),
        #               {'qM': params})
        # print(os.path.join(save_root, 'qM_'+data_file[:-4] + 'results' + '_' + args.name_predix + '_reverse.mat'))
        # print('done results')

# total_params = np.stack(total_params_sequence,axis=3)  # 128*192*192*3
# sc.io.savemat(os.path.join(root, '3params-kangning-m0sT1T2f_0_9.mat' ), {'parameters': total_params})




name:
same_as_training
root:
../../../../
same_as_training


RuntimeError: Error(s) in loading state_dict for model:
	Missing key(s) in state_dict: "fc21.weight", "fc21.bias", "bn21.weight", "bn21.bias", "bn21.running_mean", "bn21.running_var", "bn15.weight", "bn15.bias", "bn15.running_mean", "bn15.running_var", "fc16.weight", "fc16.bias". 
	size mismatch for fc20.weight: copying a param with shape torch.Size([128, 26]) from checkpoint, the shape in current model is torch.Size([128, 64]).
	size mismatch for fc4.weight: copying a param with shape torch.Size([1024, 512]) from checkpoint, the shape in current model is torch.Size([768, 512]).
	size mismatch for fc4.bias: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([768]).
	size mismatch for bn4.weight: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([768]).
	size mismatch for bn4.bias: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Si

In [45]:
with open(root+'settings_files_offline/settings_'+name+'.pkl', 'rb') as f:
        settings = pickle.load(f)
        print(settings)

{'optimizer': 'Adam', 'lr': 0.01, 'model': 'full_joint_deep3_blochsimv3p2_complexproj_0817_noprojection', 'noise_type': 'Standard', 'noise_level': 0.01, 'normalization': 'After_projection', 'namepca': '/gpfs/data/asslaenderlab/20200917_InVivo_MT_1mm_MWI_1p7mm/20201021_nonSweeping_Recos_n_Fits_Symmetric_Basis/basis_v3.2_sweep_0_std_B0_pio2_symmetric_B1_0.9pm0.2.mat', 'loss': ['MSE-CRB', 'MSE-CRB', 'MSE-CRB', 'MSE-CRB', 'MSE-CRB', 'MSE', 'MSE', 'MSE'], 'batch_size': 1024, 'start_by_projection': True, 'nb_epochs': 800, 'params': [0, 1, 2], 'initialization': 'Random', 'validation_size': 56320, 'validation': True, 'small_validation_size': 10240, 'minPD': 0.1, 'maxPD': 1.0, 'save_name': 'd0428_v3_nonsweep_varyB0B1_complex_nob1_blochsim_R13_p3_lr_0.01_noise0.01_1100file', 'nb_files': 1000, 'path_files': 'v3_uniform_b1cutoff_B0B1vary_R13_complex', 'dimension_projection': 13}


In [47]:
net['NN']

OrderedDict([('fc20.weight',
              tensor([[ 5.3813e-02, -6.8055e-01, -5.2034e-01,  ..., -3.0679e+00,
                       -1.4513e+00,  2.5092e+00],
                      [-1.1723e-01,  4.5622e-01, -7.2330e-01,  ...,  5.8514e-01,
                        2.0565e+00,  1.2188e-01],
                      [ 4.5138e-02,  6.1122e-01,  6.7482e-01,  ..., -1.8998e+00,
                       -2.6420e+00, -2.0964e+00],
                      ...,
                      [ 2.0775e-01, -7.1036e-01, -4.7315e-01,  ..., -2.7338e+00,
                        5.5104e+00,  4.2489e-01],
                      [-7.4758e-02, -1.1186e+00,  1.7372e-01,  ..., -8.1124e-01,
                        1.1067e+00,  7.7689e-01],
                      [-8.5433e-01, -9.8021e-01, -2.3737e+00,  ...,  4.5916e+00,
                       -1.6667e-03,  1.0833e+00]])),
             ('fc20.bias',
              tensor([-0.0630, -1.5069,  0.6906, -1.7012,  0.0764,  0.0229,  1.3371, -1.1476,
                       1.4557,  0.